In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
matplotlib.style.use('ggplot')

In [3]:
# pull out name into something more useful. 
import re
mrPattern = re.compile('.*Mr\..*')
missPattern = re.compile('.*Miss\..*')
masterPattern = re.compile('.*Master\..*')
mrsPattern = re.compile('.*Mrs\..*')
donPattern = re.compile('.*Don\..*')
revPattern = re.compile('.*Rev\..*')
drPattern = re.compile('.*Dr\..*')
mmePattern = re.compile('.*Mme\..*')
msPattern = re.compile('.*Ms\..*')
majorPattern = re.compile('.*Major\..*')
ladyPattern = re.compile('.*Lady\..*')
sirPattern = re.compile('.*Sir\..*')
mllePattern = re.compile('.*Mlle\..*')
colPattern = re.compile('.*Col\..*')
captPattern = re.compile('.*Capt\..*')
countessPattern = re.compile('.*Countess\..*')
jonkheerPattern = re.compile('.*Jonkheer\..*')

def nameToCategory(name):
    if (mrPattern.match(name)):
        return 'Mr'
    elif (jonkheerPattern.match(name)):
        return 'Other'
    elif (countessPattern.match(name)):
        return 'Other'
    elif (captPattern.match(name)):
        return 'Other'
    elif (missPattern.match(name)):
        return 'Miss'
    elif (masterPattern.match(name)):
        return 'Master'
    elif (mrsPattern.match(name)):
        return 'Mrs'
    elif (donPattern.match(name)):
        return 'Other'
    elif (revPattern.match(name)):
        return 'Other'
    elif (drPattern.match(name)):
        return 'Other'
    elif (mmePattern.match(name)):
        return 'Mrs'
    elif (msPattern.match(name)):
        return 'Miss'
    elif (majorPattern.match(name)):
        return 'Other'
    elif (ladyPattern.match(name)):
        return 'Other'
    elif (sirPattern.match(name)):
        return 'Other'
    elif (mllePattern.match(name)):
        return 'Miss'
    elif (colPattern.match(name)):
        return 'Other'
    return 'Other'
#    raise Exception(name)

In [4]:
import math

def scrub(filePath):
    data = pd.read_csv(filePath)
    char_cabin = data['Cabin'].astype(str)
    new_cabin = np.array([cabin[0] for cabin in char_cabin])
    data['Cabin'] = pd.Categorical(new_cabin)

    c1Median = data.Age[data.Pclass == 1].median()
    c2Median = data.Age[data.Pclass == 2].median()
    c3Median = data.Age[data.Pclass == 3].median()

    def medianFor(row):
        if (row['Pclass'] == 1):
            return c1Median
        elif (row['Pclass'] == 2):
            return c2Median
        elif (row['Pclass'] == 3):
            return c3Median
        else:
            raise Exception('Goofed')
    
    def updateAge(row):
        if (math.isnan(row['Age'])):
            median = medianFor(row)
            row['Age'] = median
        return row
    
    # Update the missing ages with the median
    data = data.apply(updateAge, axis=1)
    
    data['Title'] = data['Name'].apply(nameToCategory)
    
    
    return data

    
titanic_train = scrub('train.csv')

titanic_train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,n,S,Other
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B,S,Miss
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,24.0,1,2,W./C. 6607,23.45,n,S,Miss
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C,C,Mr
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,n,Q,Mr


In [5]:
from sklearn import linear_model
from sklearn import preprocessing

In [6]:
label_encoder = label_encoder = preprocessing.LabelEncoder()

def trainFeaturesFor(df):
    encoded_sex = label_encoder.fit_transform(df["Sex"])
    encoded_class = label_encoder.fit_transform(df["Pclass"])
    encoded_cabin = label_encoder.fit_transform(df["Cabin"])
    encoded_title = label_encoder.fit_transform(df["Title"])

    train_features = pd.DataFrame([ encoded_class
                                  , encoded_cabin
                                  , encoded_sex
                                  , encoded_title
                                  , df["Age"]
                                  ]).T
    return train_features

def trainModel(df):
    train_features = trainFeaturesFor(df)
    log_model = linear_model.LogisticRegression()
    log_model.fit( X = train_features
                 , y = titanic_train["Survived"])
    return log_model

log_model = trainModel(titanic_train)

preds = log_model.predict(X=trainFeaturesFor(titanic_train))

print(log_model.intercept_)
print(log_model.coef_)

[ 3.46866195]
[[-0.96830619 -0.05656855 -2.42177526 -0.0807839  -0.02580891]]


In [7]:
pd.crosstab(preds,titanic_train["Survived"])

Survived,0,1
row_0,,
0,469,102
1,80,240


In [8]:
log_model.score( X=trainFeaturesFor(titanic_train)
               , y=titanic_train['Survived'])

0.79573512906846244

In [9]:
titanic_test = scrub('test.csv')
test_features = trainFeaturesFor(titanic_test)
test_preds = log_model.predict(X=test_features)


In [13]:

submission = pd.DataFrame({ "PassengerId": titanic_test["PassengerId"]
                          , "Survived":test_preds})

submission.to_csv( "submission.csv"
                 , index=False)
